In [ ]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.4 MB/s eta 0:00:0000:0100:01


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from diffusers import StableDiffusionPipeline
from PIL import Image
import os
from torchvision.models import vgg19
import torch.nn.functional as nn

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load pre-trained model
pipe = StableDiffusionPipeline.from_pretrained("nota-ai/bk-sdm-small", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
#Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(512, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),  # Convert to tensor
])


In [ ]:
# Custom dataset class for image-to-image task
class ImageToImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_images = [os.path.join(input_dir, fname) for fname in os.listdir(input_dir)]
        self.target_images = [os.path.join(target_dir, fname) for fname in os.listdir(target_dir)]
        self.transform = transform

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        input_img = Image.open(self.input_images[idx]).convert("RGB")
        target_img = Image.open(self.target_images[idx]).convert("RGB")
        
        if self.transform:
            input_img = self.transform(input_img)
            target_img = self.transform(target_img)

        return input_img, target_img

In [ ]:
#Dataset
input_dir = "/kaggle/input/grayscale-to-color-dataset/input"  # Directory for grayscale images
target_dir = "/kaggle/input/grayscale-to-color-dataset/output"  # Directory for color images

dataset = ImageToImageDataset(input_dir, target_dir, transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# Freeze the parts of the model that are not relevant (e.g., text encoder)
for name, module in pipe.unet.named_modules():
    for param in module.parameters():
        param.requires_grad = False  # Freeze all parameters of UNet

# Now, unfreeze only the UNet for training
for param in pipe.unet.parameters():
    param.requires_grad = True

In [ ]:
optimizer = torch.optim.Adam(pipe.unet.parameters(), lr=0.0001)


num_epochs = 5
for epoch in range(num_epochs):
    for input_img, target_img in dataloader:
        input_img = input_img.to("cuda")
        target_img = target_img.to("cuda")

        optimizer.zero_grad()

        # Forward pass for image-to-image generation using init_image (input_img)
        prompt = "Turn this grayscale image into a vibrant colorized version, with realistic colors"
        generated_output = pipe(prompt=prompt, init_image=input_img, strength=0.6,guidance_scale=2)

   
        output_image = generated_output.images[0]  
        transform = transforms.ToTensor()
        output_tensor = transform(output_image).unsqueeze(0).to("cuda")  
        output_tensor.requires_grad = True

        # Loss function 
        loss = nn.mse_loss(output_tensor, target_img)

        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipykernel_23/3380293258.py:28: UserWarning: Using a target size (torch.Size([8, 3, 512, 512])) that is different to the input size (torch.Size([1, 3, 512, 512])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(output_tensor, target_img)


  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipykernel_23/3380293258.py:28: UserWarning: Using a target size (torch.Size([7, 3, 512, 512])) that is different to the input size (torch.Size([1, 3, 512, 512])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(output_tensor, target_img)


Epoch [1/5], Loss: 0.13667020201683044


...

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [4/5], Loss: 0.08797141909599304


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [5/5], Loss: 0.11794612556695938


In [8]:
# Save the fine-tuned model
pipe.save_pretrained("fine_tuned_model")

In [ ]:
# Testing model
test_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

test_image = Image.open("/kaggle/input/grayscale-to-color-dataset/input/img1_grayscale.jpg").convert("RGB")
test_image = test_transform(test_image).unsqueeze(0).to("cuda")

with torch.no_grad():
    output = pipe(prompt="Turn this grayscale image into a vibrant colorized version, with realistic colors", init_image=test_image, strength=0.6, guidance_scale=2) 

# Save the result
output_image = output.images[0]
output_image.save("colorized_output.png")

  0%|          | 0/50 [00:00<?, ?it/s]